In [48]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout
import keras_tuner as kt
import json
import pandas as pd
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, classification_report
import json
import matplotlib.pyplot as plt
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
pd.set_option('display.max_columns', None)


In [49]:
df = pd.read_stata('lendingclub_train.dta')
# df.head()
# df.describe()
columns_to_drop = [
    "index",
    "depvar",
    "total_acc", 
    "out_prncp", 
    "out_prncp_inv", 
    "total_pymnt", 
    "total_pymnt_inv", 
    "total_rec_prncp", 
    "total_rec_int", 
    "total_rec_late_fee", 
    "recoveries", 
    "collection_recovery_fee",
    "last_pymnt_amnt", 
    # "last_fico_range_high", 
    # "last_fico_range_low", 
    # "tot_coll_amt", 
    # "tot_cur_bal", 
    "initial_list_status1", 
    "initial_list_status2", 
    "elapsed_t",
    "purpose1",
    "addr_state1",
    "elapsed_t",
    "debt_settlement_flag1",
    "term1",
    "mths_since_last_delinq1",
    "mths_since_last_major_derog1",
    "mths_since_last_record1",
    "mths_since_rcnt_il1",
    "mths_since_recent_bc1",
    "mths_since_recent_bc_dlq1",
    "mths_since_recent_inq1",
    "mths_since_recent_revol_delinq1"
]

issue_d_count = 1
while(issue_d_count <= 118):
    word_tmp = "issue_d" + str(issue_d_count)
    columns_to_drop.append(word_tmp)
    issue_d_count += 1


### Undersampling Logic
Undersampling을 안할거면 다음 로직 실행

In [ ]:
# Separate majority and minority classes
df_majority = df[df['depvar'] == 0]
df_minority = df[df['depvar'] == 1]

# Count number of instances in the minority class
minority_count = len(df_minority)

# Undersample the majority class
df_majority_undersampled = df_majority.sample(n=minority_count*2, random_state=42)

# Combine minority class with the undersampled majority class
df_undersampled = pd.concat([df_majority_undersampled, df_minority])

# Shuffle the dataset
df_undersampled = df_undersampled.sample(frac=1, random_state=42)

# Now split into features and target, and then into training and testing sets
y = df_undersampled['depvar']
X = df_undersampled.drop(columns=columns_to_drop)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# X.info(verbose=True,max_cols=None)
# y.head()
X.head()


In [50]:
y = df['depvar']
X = df.drop(columns=columns_to_drop)
# X = df.drop(columns=['depvar','index'])

X.info(verbose=True,max_cols=None)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 874335 entries, 0 to 874334
Data columns (total 188 columns):
 #    Column                            Dtype  
---   ------                            -----  
 0    loan_amnt                         int32  
 1    funded_amnt                       int32  
 2    funded_amnt_inv                   float64
 3    int_rate                          float64
 4    installment                       float64
 5    annual_inc                        float64
 6    dti                               float64
 7    delinq_2yrs                       int8   
 8    fico_range_low                    int16  
 9    fico_range_high                   int16  
 10   inq_last_6mths                    int8   
 11   open_acc                          int16  
 12   pub_rec                           int8   
 13   revol_bal                         int32  
 14   revol_util                        float64
 15   last_fico_range_high              int16  
 16   last_fico_range_lo

# Hyperparameter Tuning

In [51]:
# Split the dataset into tuning subset and remaining data
X_tuning, X_remaining, y_tuning, y_remaining = train_test_split(X, y, test_size=0.8, random_state=42)  # 20% for tuning

# Further split the tuning subset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_tuning, y_tuning, test_size=0.2, random_state=42)

# Compute class weights for the tuning subset
class_weights = compute_class_weight('balanced', classes=[0, 1], y=y_train)
class_weight_dict = dict(enumerate(class_weights))

def build_model(hp):
    model = Sequential()
    model.add(Dense(units=hp.Int('units', min_value=32, max_value=512, step=32), activation='relu', input_shape=(X_train.shape[1],)))
    model.add(Dropout(rate=hp.Float('dropout', min_value=0, max_value=0.5, step=0.1)))
    model.add(Dense(units=hp.Int('units2', min_value=32, max_value=512, step=32), activation='relu'))
    model.add(Dropout(rate=hp.Float('dropout2', min_value=0, max_value=0.5, step=0.1)))
    model.add(Dense(units=hp.Int('units3', min_value=32, max_value=512, step=32), activation='relu'))
    model.add(Dropout(rate=hp.Float('dropout3', min_value=0, max_value=0.5, step=0.1)))
    model.add(Dense(1, activation='sigmoid'))  # Single output neuron

    model.compile(optimizer=tf.keras.optimizers.legacy.Adam(hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
                  loss='binary_crossentropy',  # Binary crossentropy loss
                  metrics=['accuracy'])

    return model

tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=10,
    directory='hyperband',
    project_name='hyperband_tuning'
)

tuner.search(X_train, y_train, epochs=10, validation_data=(X_val, y_val), class_weight=class_weight_dict)

# Get the best model
best_model = tuner.get_best_models(num_models=1)[0]

# Evaluate the best model
loss, accuracy = best_model.evaluate(X_val, y_val)
print(f"Test Accuracy: {accuracy:.2f}, Test Loss: {loss:.2f}")

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Prepare the best hyperparameters for saving
best_hyperparameters = {
    'units': best_hps.get('units'),
    'units2': best_hps.get('units2'),
    'units3': best_hps.get('units3'),
    'dropout': best_hps.get('dropout'),
    'dropout2': best_hps.get('dropout2'),
    'dropout3': best_hps.get('dropout3'),
    'learning_rate': best_hps.get('learning_rate')
}

# Save the best hyperparameters to a JSON file
with open('best_hyperparameters.json', 'w') as f:
    json.dump(best_hyperparameters, f)

print("Best hyperparameters saved to 'best_hyperparameters.json'")

Trial 30 Complete [00h 01m 23s]
val_accuracy: 0.8357351422309875

Best val_accuracy So Far: 0.8357351422309875
Total elapsed time: 00h 17m 34s
1093/1093 [==============================] - 1s 786us/step - loss: 0.6925 - accuracy: 0.8357
Test Accuracy: 0.84, Test Loss: 0.69
Best hyperparameters saved to 'best_hyperparameters.json'


# Model Training

In [53]:
X_train, X_test, y_train, y_test =  train_test_split(X_remaining, y_remaining, test_size=0.2, random_state=42)

# Class Weight
class_weights = compute_class_weight('balanced', classes=[0,1], y=y_train)
class_weight_dict = dict(enumerate(class_weights))


# Load best hyperparameters
with open('best_hyperparameters.json') as f:
    best_hps = json.load(f)

model_path = 'final_trained_model.h5'
if(os.path.exists(model_path)):
    model = load_model(model_path)
else:
    model = Sequential()
    model.add(Dense(units=best_hps['units'], activation='relu', input_shape=(X_train.shape[1],)))
    model.add(Dropout(best_hps['dropout']))
    model.add(Dense(units=best_hps['units2'], activation='relu'))
    model.add(Dropout(best_hps['dropout2']))
    model.add(Dense(units=best_hps['units3'], activation='relu'))
    model.add(Dropout(best_hps['dropout3']))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer=tf.keras.optimizers.legacy.Adam(0.001),
                loss='binary_crossentropy',
                metrics=['accuracy'])

history = model.fit(X, y, epochs=200, batch_size=128, class_weight=class_weight_dict)  # Adjust epochs and batch_size as needed

# Save the final trained model
model.save('final_trained_model.h5')


Epoch 1/200
6831/6831 [==============================] - 25s 4ms/step - loss: 0.6936 - accuracy: 0.4914
Epoch 2/200
6831/6831 [==============================] - 25s 4ms/step - loss: 0.6936 - accuracy: 0.4305
Epoch 3/200
6831/6831 [==============================] - 24s 4ms/step - loss: 0.6936 - accuracy: 0.4811
Epoch 4/200
6831/6831 [==============================] - 24s 4ms/step - loss: 0.6936 - accuracy: 0.4663
Epoch 5/200
6831/6831 [==============================] - 25s 4ms/step - loss: 0.6936 - accuracy: 0.4573
Epoch 6/200
6831/6831 [==============================] - 25s 4ms/step - loss: 0.6936 - accuracy: 0.4769
Epoch 7/200
6831/6831 [==============================] - 25s 4ms/step - loss: 0.6936 - accuracy: 0.4413
Epoch 8/200
6831/6831 [==============================] - 25s 4ms/step - loss: 0.6936 - accuracy: 0.4713
Epoch 9/200
6831/6831 [==============================] - 24s 4ms/step - loss: 0.6936 - accuracy: 0.4717
Epoch 10/200
6831/6831 [==============================] - 24s 4m

KeyboardInterrupt: 

### Plot the learning 

In [ ]:

# Plotting the training history
plt.plot(history.history['accuracy'], label='accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.title('Training Accuracy')
plt.show()

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f"Test accuracy: {test_accuracy}")

# Predictions for confusion matrix
y_pred = model.predict(X_test)
y_pred = np.round(y_pred).astype(int).reshape(-1)  # Adjust this line if not a binary classification

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", conf_matrix)

# Precision, Recall, F1-Score
report = classification_report(y_test, y_pred)
print("Classification Report:\n", report)